# Laboratório: Aprender a ler dados RAW com PySpark

## 1. Instale as bibliotecas necessárias (se ainda não instalou)


In [ ]:
!pip install minio pandas pyarrow pyspark

## 2. Importe as bibliotecas

In [ ]:
from minio import Minio
import pandas as pd
import io

## 3. Configure as credenciais e endpoint do MinIO

In [ ]:
minio_client = Minio(
    "minio:9000",
    access_key="4PRJYFLGzQYTnOJGH1gA",
    secret_key="ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD",
    secure=False
)

## 4. Liste os arquivos Parquet do bucket 'raw'


In [ ]:
# Lista todos os arquivos Parquet disponíveis no bucket 'raw' do MinIO.
bucket_name = "raw"
objects = minio_client.list_objects(bucket_name, recursive=True)
parquet_files = [obj.object_name for obj in objects if obj.object_name.endswith('.parquet')]
print(parquet_files)

In [ ]:
# Para cada arquivo Parquet (tabela), leia os dados em um DataFrame e armazene em um dicionário.
tables = {}
for file in parquet_files:
    response = minio_client.get_object(bucket_name, file)
    data = response.read()
    df = pd.read_parquet(io.BytesIO(data))
    #df.drop_duplicates(inplace=True)  # Remove duplicatas, se necessário
    #df.dropna(inplace=True)  # Remove linhas com valores ausentes, se necessário
    table_name = file.split('/')[-2] # Extrai o nome da tabela do caminho do arquivo
    tables[table_name] = df
    response.close()
    response.release_conn()

# Agora, 'tables' é um dicionário onde a chave é o nome da tabela e o valor é o DataFrame correspondente.
tables['film']

## 5. Criando um Warehouse no LakeKeeper

In [ ]:
import requests

# Definindo a URL base da API do Lakekeeper
CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
WAREHOUSE = "trusted"

response = requests.post(
    f"{MANAGEMENT_URL}/v1/warehouse",
    json={
        "warehouse-name": WAREHOUSE,
        "storage-profile": {
            "type": "s3",
            "bucket": "trusted",
            "flavor": "minio",
            "key-prefix": None,
            "assume-role-arn": None,
            "endpoint": "http://minio:9000",
            "region": "local-01",
            "path-style-access": True,
            "sts-enabled": False
        },
        "storage-credential": {
            "type": "s3",
            "credential-type": "access-key",
            "aws-access-key-id": "4PRJYFLGzQYTnOJGH1gA",
            "aws-secret-access-key": "ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD"
        }
    }
)
print(f"{response.status_code}: {response.reason}")

## 6. Criando uma tabela com PySpark e Lakekeeper

In [ ]:
# Configuração do Spark para integração com o catálogo Iceberg via REST, utilizando o Lakekeeper e MinIO.
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.0"
CATALOG = "trusted"

config = {
    "spark.sql.defaultCatalog": "trusted",
    f"spark.sql.catalog.{CATALOG}": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.{CATALOG}.type": "rest",
    f"spark.sql.catalog.{CATALOG}.uri": CATALOG_URL,
    f"spark.sql.catalog.{CATALOG}.warehouse": WAREHOUSE,
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.jars.packages": f"""org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}"""
}

print(config)
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

spark.sql(f"USE {CATALOG}")

In [ ]:
# Criação do namespace (banco de dados) 'pagila_db' no catálogo Iceberg, caso não exista.
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS pagila_db")
spark.sql("SHOW NAMESPACES").toPandas()

## 7. Agrupando as partições de pagamento

In [ ]:
# Cria uma lista chamada payment_dfs para armazenar os DataFrames relacionados a pagamentos
payment_dfs = []

# Percorre todas as tabelas carregadas
for nome_tabela, df in tables.items():
    # Se o nome da tabela contém a palavra 'payment', adiciona o DataFrame à lista
    if 'payment' in nome_tabela:
        payment_dfs.append(df)

# Cria um DataFrame Spark a partir do DataFrame concatenado e escreve na tabela 'pagila_db.payments'
if payment_dfs:
    payments_df = pd.concat(payment_dfs, ignore_index=True)
    sdf = spark.createDataFrame(payments_df)
    sdf.writeTo("pagila_db.payments").createOrReplace()

## 8. Criando as demais tabelas

In [ ]:
# Escreve as demais tabelas no catálogo Iceberg, exceto as já processadas de pagamento.
for table_name, df in tables.items():
    if 'payment' in table_name:
        continue  # já processado acima
    if not df.empty:
        sdf = spark.createDataFrame(df)
        sdf.writeTo(f"pagila_db.{table_name}").createOrReplace()
    else:
        print(f"Skipping empty DataFrame for table: {table_name}")